In [3]:
import os
import sys
# to import the files in path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.orthanc_scrapper import *
from src.kz.kz_bi import *
logging.getLogger('WDM').setLevel(logging.NOTSET)

In [4]:
base_flat_url = 'https://bi.group/ru/flats?placementUUID='
main_url = 'https://bi.group/ru/filter?propertyTypes=%5B%225990a172-812a-4fee-b4f5-c860cca824d7%22%2C%22b6e20785-9b33-46a9-86b5-707fdbffe314%22%2C%22a6deff39-99d2-4a4c-982c-b245b7e43037%22%2C%22b3be088f-52d2-47af-93d5-0667312547c9%22%2C%22eb845125-c2b7-4d8a-93d7-015080355f78%22%2C%22c2c7b7b0-6b3e-4b9a-b729-64a750fe271d%22%2C%2264d2ff0a-22d9-4fa3-92aa-6391faf460f9%22%2C%22e211be72-2986-4ea1-8991-bfe7233ac4c2%22%2C%22253c179e-7a74-4096-a0e9-a63df0e24bb5%22%2C%228f72b6b1-0453-4938-9775-0f2f3a836ccd%22%5D&realEstatePage=%22PLACEMENTS%22&realEstateUUIDs=%5B%22c68f11e7-48d1-11eb-a83d-00155d106579%22%5D&cityUUID=%224c0fe725-4b6f-11e8-80cf-bb580b2abfef%22&floorMin=1&floorMax=12'
country = 'kz'
file_name = 'bi_aqua'
flats_characteristics = pd.DataFrame(columns=['Floor','Number Of Floors', 'Surface', 'Price', 'Entrance', 'Link'])
bi = KzBIGroup(main_url, base_flat_url, country, file_name, flats_characteristics)
bi.find_all_flats_urls_on_main_page()
#bi.find_flats_characteristics()

['https://bi.group/ru/flats?placementUUID=0996a343-5978-11ec-a829-001dd8b726aa',
 'https://bi.group/ru/flats?placementUUID=b524dea1-599f-11ec-a829-001dd8b726aa',
 'https://bi.group/ru/flats?placementUUID=0b341d42-597f-11ec-a829-001dd8b726aa',
 'https://bi.group/ru/flats?placementUUID=6f782479-5977-11ec-a829-001dd8b726aa',
 'https://bi.group/ru/flats?placementUUID=9db21e85-614e-11ec-a81f-001dd8b72708',
 'https://bi.group/ru/flats?placementUUID=b113b433-614e-11ec-a81f-001dd8b72708',
 'https://bi.group/ru/flats?placementUUID=834bb6f1-5980-11ec-a829-001dd8b726aa',
 'https://bi.group/ru/flats?placementUUID=f61253e4-597e-11ec-a829-001dd8b726aa',
 'https://bi.group/ru/flats?placementUUID=93103845-5980-11ec-a829-001dd8b726aa']

In [5]:
bi.find_flats_characteristics()

,Floor,Number Of Floors,Surface,Price,Entrance,Link
0,4,12,40.91,25314739,5,https://bi.group/ru/flats?placementUUID=0996a3...
0,9,9,46.24,25698803,7,https://bi.group/ru/flats?placementUUID=b524de...
0,2,9,46.85,26039230,6,https://bi.group/ru/flats?placementUUID=0b341d...
0,9,9,45.81,25458419,4,https://bi.group/ru/flats?placementUUID=6f7824...
0,4,12,41.39,25611759,3,https://bi.group/ru/flats?placementUUID=9db21e...
0,6,12,41.39,25611759,3,https://bi.group/ru/flats?placementUUID=b113b4...
0,9,9,42.82,23583879,6,https://bi.group/ru/flats?placementUUID=834bb6...
0,2,9,42.83,23803525,6,https://bi.group/ru/flats?placementUUID=f61253...
0,9,9,46.22,25689076,6,https://bi.group/ru/flats?placementUUID=931038...


In [6]:
bi.flats_characteristics.groupby(['Entrance']).size().reset_index(name='counts')

,Entrance,counts
0,1,22
1,2,6
2,3,13
3,4,31
4,5,18
5,6,19
6,7,37


In [31]:
floor = '9 из 9'
floor, max_floor = floor.split(' из ')

In [19]:
#f = bi.flats_characteristics.copy()
#f[['Floor','NumberOfFloors']]=f['Floor'].str.split(' из ', 1, expand=True)
f['NumberOfFloors']=pd.to_numeric(f['NumberOfFloors'])
f['Floor']=pd.to_numeric(f['Floor'])

In [29]:
intersting_flats = f.loc[(f['NumberOfFloors']==9) & (f['Floor'].isin([4,5,6,7]))]
intersting_flats = pd.concat([intersting_flats, f.loc[(f['NumberOfFloors']==12) & (f['Floor'].isin([4,5,6,7,9]))]])
intersting_flats.groupby(['Entrance']).size().reset_index(name='counts')

,Entrance,counts
0,1,9
1,2,2
2,3,6
3,4,14
4,5,8
5,6,7
6,7,18


In [133]:
import datetime
import logging
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


SCRAPING_TIMEOUT = 30
URL_FILE_PATH = 'C:/dev/re_kz/bi_urls.txt'

logging.getLogger('WDM').setLevel(logging.NOTSET)

def get_selenium_scraping_options():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('start-maximized')
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_experimental_option('useAutomationExtension', False)
    return options


def get_webdriver():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=get_selenium_scraping_options())
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {'userAgent': user_agent})
    return driver

def analyse_url(driver, url):
    #driver = get_webdriver()
    driver.get(url)
    element_price = WebDriverWait(driver, SCRAPING_TIMEOUT).until(
                    EC.presence_of_element_located(
                        (By.XPATH, "//div[contains(text(),'Стоимость')]//following::div[1]")
                    )
                )
    price = int(element_price.text.replace(' ₸','').replace(",",""))

    element_etage = WebDriverWait(driver, SCRAPING_TIMEOUT).until(
                    EC.presence_of_element_located(
                        (By.XPATH, "//div[contains(text(),'Этаж')]//following::div[1]")
                    )
                )
    etage = element_etage.text

    element_surface = WebDriverWait(driver, SCRAPING_TIMEOUT).until(
                    EC.presence_of_element_located(
                        (By.XPATH, "//div[contains(text(),'Площадь')]//following::div[1]")
                    )
                )
    surface = element_surface.text.replace(' м²','')
    
    element_entrance = WebDriverWait(driver, SCRAPING_TIMEOUT).until(
                    EC.presence_of_element_located(
                        (By.XPATH, "//div[contains(text(),'Подъезд')]//following::div[1]")
                    )
                )
    entrance = element_entrance.text
    return pd.DataFrame([[etage, surface, price,entrance,url]],columns=['Floor','Surface','Price','Entrance', 'Link'])

def read_urls():
    urls = []
    with open(URL_FILE_PATH) as url_file:
        for line in url_file:
            urls.append(str(line)[:-1])
    return urls

def analyze_urls_old():
    urls = read_urls()
    result = pd.DataFrame(columns=['Floor','Surface','Price','Entrance', 'Link'])
    for url in urls:
        elements = analyse_url(driver, url)
        result= pd.concat([result,elements])
    result= result.reset_index(drop=True)
    today = datetime.datetime.today().strftime('%Y-%m-%d')
    result.to_csv('C:/dev/re_kz/'+today+'bi_analysis.csv')
    return result

def click_button(driver, button_class_to_find):
    #button = driver.find_element_by_xpath(button_class_to_find)
    button = get_by_path(driver, button_class_to_find)
    driver.execute_script("arguments[0].click();", button)
    
def load_more(driver):
    button_class_to_find="//button[starts-with(@class, 'MRE-MuiButtonBase-root MRE-MuiButton-root MRE-MuiButton-contained MRE-jss')]//span[text()='Показать еще']"
    click_button(driver, button_class_to_find)

def get_by_path(driver, element_to_look_for):
    return WebDriverWait(driver, SCRAPING_TIMEOUT).until(
                    EC.presence_of_element_located(
                        (By.XPATH, element_to_look_for)
                    )
                )

def analyze_urls():
    driver = get_webdriver()
    base_url_flat = 'https://bi.group/ru/flats?placementUUID='
    flat_ids = find_all_flat_ids(driver, main_url)
    result = pd.DataFrame(columns=['Floor','Surface','Price','Entrance', 'Link'])
    for flat_id in flat_ids:
        url = base_url_flat + flat_id
        elements = analyse_url(driver, url)
        result= pd.concat([result,elements])
    result= result.reset_index(drop=True)
    today = datetime.datetime.today().strftime('%Y-%m-%d')
    result.to_csv('C:/dev/re_kz/'+today+'bi_analysis.csv')
    return result

def find_all_flat_ids(driver, main_url):
    driver.get(main_url)
    n_ids_prev = 0
    uids = []
    try:
        load_more(driver)
    except Exception as e:
        print("Exception before while loop")
        print(e)
        
    driver.implicitly_wait(2)
    element_urls = driver.find_elements_by_xpath("//img[starts-with(@class,'MRE-jss')]")
    #print("element_urls",element_urls)
    for element_url in element_urls:
        uid = element_url.get_attribute("src").split("/")[-2]
        uids.append(uid)
    uids = list(set(uids))
    n_ids = len(uids)
    while n_ids>n_ids_prev:
        try:
            load_more(driver)
        except Exception as e:
            print("Exception in While Loop")
            print(e)
        element_urls = driver.find_elements_by_xpath("//img[@class='MRE-jss134']")
        for element_url in element_urls:
            uid = element_url.get_attribute("src").split("/")[-2]
            uids.append(uid)
        uids= list(set(uids))
        n_ids_prev = n_ids
        n_ids = len(uids)
    return uids

In [134]:
analyze_urls()

C:\Users\arthu\AppData\Local\Temp/ipykernel_16984/2111990143.py:127: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  element_urls = driver.find_elements_by_xpath("//img[starts-with(@class,'MRE-jss')]")
C:\Users\arthu\AppData\Local\Temp/ipykernel_16984/2111990143.py:140: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  element_urls = driver.find_elements_by_xpath("//img[@class='MRE-jss134']")


,Floor,Surface,Price,Entrance,Link
0,9 из 9,45.81,25245891,4,https://bi.group/ru/flats?placementUUID=6f7824...
1,9 из 9,42.82,23384002,6,https://bi.group/ru/flats?placementUUID=834bb6...


In [122]:

driver = get_webdriver()
base_url_flat = 'https://bi.group/ru/flats?placementUUID='
flat_ids = find_all_flat_ids(driver, main_url)
print(flat_ids)
for flat_id in flat_ids:
    url = base_url_flat + flat_id
    print(url)

C:\Users\arthu\AppData\Local\Temp/ipykernel_16984/3546373257.py:121: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  element_urls = driver.find_elements_by_xpath("//img[starts-with(@class,'MRE-jss')]")


[<selenium.webdriver.remote.webelement.WebElement (session="3b7580dcbd78a94e94c27da5b70c37cf", element="d13bea3f-6f2c-4f2c-950a-9773abd7188a")>, <selenium.webdriver.remote.webelement.WebElement (session="3b7580dcbd78a94e94c27da5b70c37cf", element="615b18b5-219c-4a6a-88a3-8a099446b898")>, <selenium.webdriver.remote.webelement.WebElement (session="3b7580dcbd78a94e94c27da5b70c37cf", element="ed901f6f-5156-424e-ba93-7ef844bfa3d7")>, <selenium.webdriver.remote.webelement.WebElement (session="3b7580dcbd78a94e94c27da5b70c37cf", element="2e95f1d8-c831-4c75-abd6-76d4c172638e")>, <selenium.webdriver.remote.webelement.WebElement (session="3b7580dcbd78a94e94c27da5b70c37cf", element="38594682-82be-481d-a41c-c29fb19837e5")>, <selenium.webdriver.remote.webelement.WebElement (session="3b7580dcbd78a94e94c27da5b70c37cf", element="8db193d6-6921-49a5-abb6-42df3bfa0ca6")>, <selenium.webdriver.remote.webelement.WebElement (session="3b7580dcbd78a94e94c27da5b70c37cf", element="b549765f-b6e2-4876-929b-194d5474

In [99]:
main_url = 'https://bi.group/ru/filter?propertyTypes=%5B%225990a172-812a-4fee-b4f5-c860cca824d7%22%2C%22b6e20785-9b33-46a9-86b5-707fdbffe314%22%2C%22a6deff39-99d2-4a4c-982c-b245b7e43037%22%2C%22b3be088f-52d2-47af-93d5-0667312547c9%22%2C%22eb845125-c2b7-4d8a-93d7-015080355f78%22%2C%22c2c7b7b0-6b3e-4b9a-b729-64a750fe271d%22%2C%2264d2ff0a-22d9-4fa3-92aa-6391faf460f9%22%2C%22e211be72-2986-4ea1-8991-bfe7233ac4c2%22%2C%22253c179e-7a74-4096-a0e9-a63df0e24bb5%22%2C%228f72b6b1-0453-4938-9775-0f2f3a836ccd%22%5D&realEstatePage=%22PLACEMENTS%22&floorMax=10&realEstateUUIDs=%5B%22c68f11e7-48d1-11eb-a83d-00155d106579%22%5D&cityUUID=%224c0fe725-4b6f-11e8-80cf-bb580b2abfef%22&floorMin=3'

driver = get_webdriver()
driver.get(main_url)


<selenium.webdriver.remote.webelement.WebElement (session="4cf353ea5489b9dfcade816f8f345f1d", element="1679d3c2-b084-4c2c-8318-3ab34e1e61c4")>

In [89]:
find_all_flat_urls(main_url)

C:\Users\arthu\AppData\Local\Temp/ipykernel_16984/251290119.py:87: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = driver.find_element_by_xpath(button_class_to_find)
C:\Users\arthu\AppData\Local\Temp/ipykernel_16984/251290119.py:100: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  element_urls = driver.find_elements_by_xpath("//img[@class='MRE-jss134']")


loading


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[starts-with(@class, 'MRE-MuiButtonBase-root MRE-MuiButton-root MRE-MuiButton-contained MRE-jss')]//span[text()='Показать еще']"}
  (Session info: headless chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B3DF13+2219795]
	Ordinal0 [0x00AD2841+1779777]
	Ordinal0 [0x009E423D+803389]
	Ordinal0 [0x00A13025+995365]
	Ordinal0 [0x00A131EB+995819]
	Ordinal0 [0x00A40F52+1183570]
	Ordinal0 [0x00A2E844+1108036]
	Ordinal0 [0x00A3F192+1175954]
	Ordinal0 [0x00A2E616+1107478]
	Ordinal0 [0x00A07F89+950153]
	Ordinal0 [0x00A08F56+954198]
	GetHandleVerifier [0x00E32CB2+3040210]
	GetHandleVerifier [0x00E22BB4+2974420]
	GetHandleVerifier [0x00BD6A0A+565546]
	GetHandleVerifier [0x00BD5680+560544]
	Ordinal0 [0x00AD9A5C+1808988]
	Ordinal0 [0x00ADE3A8+1827752]
	Ordinal0 [0x00ADE495+1827989]
	Ordinal0 [0x00AE80A4+1867940]
	BaseThreadInitThunk [0x7506FA29+25]
	RtlGetAppContainerNamedObjec

In [96]:
main_url = 'https://bi.group/ru/filter?propertyTypes=%5B%225990a172-812a-4fee-b4f5-c860cca824d7%22%2C%22b6e20785-9b33-46a9-86b5-707fdbffe314%22%2C%22a6deff39-99d2-4a4c-982c-b245b7e43037%22%2C%22b3be088f-52d2-47af-93d5-0667312547c9%22%2C%22eb845125-c2b7-4d8a-93d7-015080355f78%22%2C%22c2c7b7b0-6b3e-4b9a-b729-64a750fe271d%22%2C%2264d2ff0a-22d9-4fa3-92aa-6391faf460f9%22%2C%22e211be72-2986-4ea1-8991-bfe7233ac4c2%22%2C%22253c179e-7a74-4096-a0e9-a63df0e24bb5%22%2C%228f72b6b1-0453-4938-9775-0f2f3a836ccd%22%5D&realEstatePage=%22PLACEMENTS%22&floorMax=10&realEstateUUIDs=%5B%22c68f11e7-48d1-11eb-a83d-00155d106579%22%5D&cityUUID=%224c0fe725-4b6f-11e8-80cf-bb580b2abfef%22&floorMin=3'

driver = get_webdriver()
driver.get(main_url)
load_more(driver)
load_more(driver)

C:\Users\arthu\AppData\Local\Temp/ipykernel_16984/251290119.py:87: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = driver.find_element_by_xpath(button_class_to_find)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[starts-with(@class, 'MRE-MuiButtonBase-root MRE-MuiButton-root MRE-MuiButton-contained MRE-jss')]//span[text()='Показать еще']"}
  (Session info: headless chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B3DF13+2219795]
	Ordinal0 [0x00AD2841+1779777]
	Ordinal0 [0x009E423D+803389]
	Ordinal0 [0x00A13025+995365]
	Ordinal0 [0x00A131EB+995819]
	Ordinal0 [0x00A40F52+1183570]
	Ordinal0 [0x00A2E844+1108036]
	Ordinal0 [0x00A3F192+1175954]
	Ordinal0 [0x00A2E616+1107478]
	Ordinal0 [0x00A07F89+950153]
	Ordinal0 [0x00A08F56+954198]
	GetHandleVerifier [0x00E32CB2+3040210]
	GetHandleVerifier [0x00E22BB4+2974420]
	GetHandleVerifier [0x00BD6A0A+565546]
	GetHandleVerifier [0x00BD5680+560544]
	Ordinal0 [0x00AD9A5C+1808988]
	Ordinal0 [0x00ADE3A8+1827752]
	Ordinal0 [0x00ADE495+1827989]
	Ordinal0 [0x00AE80A4+1867940]
	BaseThreadInitThunk [0x7506FA29+25]
	RtlGetAppContainerNamedObjec

In [95]:
load_more(driver)

C:\Users\arthu\AppData\Local\Temp/ipykernel_16984/251290119.py:87: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = driver.find_element_by_xpath(button_class_to_find)


In [ ]:
# Find all the placementUUID
# then construct url with it

base_url_flat = 'https://bi.group/ru/flats?placementUUID='
'28c044c0-5978-11ec-a829-001dd8b726aa'
url_flat = base_url_flat + placementUUID

In [67]:
x="//button[@class='MRE-MuiButtonBase-root MRE-MuiButton-root MRE-MuiButton-contained MRE-jss85']//span[@class='MRE-MuiTouchRipple-root']"
elem = driver.find_element_by_xpaths(x)
driver.execute_script("arguments[0].click();", elem)

C:\Users\arthu\AppData\Local\Temp/ipykernel_16984/1231465475.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  elem = driver.find_element_by_xpath(x)


In [82]:

x="//button[starts-with(@class, 'MRE-MuiButtonBase-root MRE-MuiButton-root MRE-MuiButton-contained MRE-jss')]//span[text()='Показать еще']"
elem = driver.find_elements_by_xpath(x)[0]
driver.execute_script("arguments[0].click();", elem)

C:\Users\arthu\AppData\Local\Temp/ipykernel_16984/2664315482.py:3: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  elem = driver.find_elements_by_xpath(x)[0]


In [81]:
driver.find_elements_by_xpath("//button[starts-with(@class, 'MRE-MuiButtonBase-root MRE-MuiButton-root MRE-MuiButton-contained MRE-jss')]//span[text()='Показать еще']")[0]

C:\Users\arthu\AppData\Local\Temp/ipykernel_16984/1487263709.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  driver.find_elements_by_xpath("//button[starts-with(@class, 'MRE-MuiButtonBase-root MRE-MuiButton-root MRE-MuiButton-contained MRE-jss')]//span[text()='Показать еще']")[0]


<selenium.webdriver.remote.webelement.WebElement (session="b3f9621eccad154b346f9226278a0765", element="14749da6-c819-4abf-b856-d7b71277a806")>

In [50]:
element_urls = driver.find_elements_by_xpath("//img[@class='MRE-jss134']")
uids = []
for element_url in element_urls:
    uid = element_url.get_attribute("src").split("/")[-2]
    uids.append(uid)
uids= list(set(uids))
uids

C:\Users\arthu\AppData\Local\Temp/ipykernel_16984/2380302151.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  element_urls = driver.find_elements_by_xpath("//img[@class='MRE-jss134']")


['93103845-5980-11ec-a829-001dd8b726aa',
 'a391e036-58ed-11ec-a81f-001dd8b72708',
 '99c02188-5975-11ec-a829-001dd8b726aa',
 'c65b1503-58ed-11ec-a81f-001dd8b72708',
 '6f782479-5977-11ec-a829-001dd8b726aa',
 '42a63b63-599e-11ec-a829-001dd8b726aa',
 'b524dea1-599f-11ec-a829-001dd8b726aa',
 '53b61d38-599e-11ec-a829-001dd8b726aa',
 'b6bf8687-5976-11ec-a829-001dd8b726aa',
 'b113b433-614e-11ec-a81f-001dd8b72708',
 'cc66d446-5976-11ec-a829-001dd8b726aa',
 '11bcc9c4-5980-11ec-a829-001dd8b726aa',
 '0996a343-5978-11ec-a829-001dd8b726aa',
 '078a1225-5980-11ec-a829-001dd8b726aa',
 '19de357c-5980-11ec-a829-001dd8b726aa',
 'ebf66800-597f-11ec-a829-001dd8b726aa',
 '2d9acbcb-599a-11ec-a829-001dd8b726aa',
 '9db21e85-614e-11ec-a81f-001dd8b72708',
 '28c044c0-5978-11ec-a829-001dd8b726aa',
 '2a5cad0b-5980-11ec-a829-001dd8b726aa',
 'b4689279-58ed-11ec-a81f-001dd8b72708',
 'd3da33df-5976-11ec-a829-001dd8b726aa',
 '835cd9e7-5977-11ec-a829-001dd8b726aa',
 '834bb6f1-5980-11ec-a829-001dd8b726aa',
 'c177350d-599f-

In [29]:
element_urls = driver.find_elements_by_xpath("//img[@class='MRE-jss134']")
uids = []
for element_url in element_urls:
    uid = element_url.get_attribute("src").split("/")[-2]
    uids.append(uid)
uids= list(set(uids))
uids

C:\Users\arthu\AppData\Local\Temp/ipykernel_16984/2380302151.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  element_urls = driver.find_elements_by_xpath("//img[@class='MRE-jss134']")


['9db21e85-614e-11ec-a81f-001dd8b72708',
 '0996a343-5978-11ec-a829-001dd8b726aa',
 '28c044c0-5978-11ec-a829-001dd8b726aa',
 'b524dea1-599f-11ec-a829-001dd8b726aa',
 '93103845-5980-11ec-a829-001dd8b726aa',
 'd3da33df-5976-11ec-a829-001dd8b726aa',
 '834bb6f1-5980-11ec-a829-001dd8b726aa',
 'b113b433-614e-11ec-a81f-001dd8b72708',
 '6f782479-5977-11ec-a829-001dd8b726aa']

In [111]:
driver.page_source

'<html class="popmechanic-desktop popmechanic-landscape popmechanic-loaded"><head><script id="emarsys_webpersonalization_es6" src="//static.scarabresearch.com/wpjs/wpes6.js?ts=2753" charset="utf-8"></script><script id="emarsys_webpersonalization_loader" src="//static.scarabresearch.com/wpjs/wploader.js?ts=2753" charset="utf-8"></script><script async="" src="https://www.clarity.ms/eus/s/0.6.42/clarity.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-65H5JL9DR2&amp;l=dataLayer&amp;cx=c"></script><script type="text/javascript" async="" src="https://analytics.tiktok.com/i18n/pixel/events.js?sdkid=CAT7VPRC77UC35MGC7N0&amp;lib=ttq"></script><script async="" src="https://mc.yandex.ru/metrika/tag.js"></script><script id="scarab-js-api" src="//cdn.scarabresearch.com/js/14299F198892F844/scarab-v2.js"></script><script async="" src="https://mc.yandex.ru/metrika/tag.js"></script><script type="text/javascript" async="" src="https://www.clarity.m

In [5]:
result = analyze_urls()
result

,Floor,Surface,Price,Entrance,Link
0,7 из 12,40.91,25314739,5,https://bi.group/ru/flats?placementUUID=28c044...
1,4 из 12,40.91,25314739,5,https://bi.group/ru/flats?placementUUID=0996a3...
2,6 из 12,41.39,25611759,3,https://bi.group/ru/flats?placementUUID=b113b4...
3,4 из 12,41.39,25611759,3,https://bi.group/ru/flats?placementUUID=9db21e...
4,7 из 9,46.22,26229850,6,https://bi.group/ru/flats?placementUUID=21d255...
5,6 из 9,46.22,26229850,6,https://bi.group/ru/flats?placementUUID=19de35...
6,6 из 9,42.82,26272340,6,https://bi.group/ru/flats?placementUUID=ebf668...
7,4 из 9,46.22,26484060,6,https://bi.group/ru/flats?placementUUID=078a12...
8,5 из 9,46.24,26495520,7,https://bi.group/ru/flats?placementUUID=339491...
9,7 из 9,45.81,28106863,4,https://bi.group/ru/flats?placementUUID=cc66d4...


In [3]:
read_urls()

['https://bi.group/ru/flats?placementUUID=28c044c0-5978-11ec-a829-001dd8b726aa&realEstateUUID=c68f11e7-48d1-11eb-a83d-00155d106579',
 'https://bi.group/ru/flats?placementUUID=0996a343-5978-11ec-a829-001dd8b726aa&realEstateUUID=c68f11e7-48d1-11eb-a83d-00155d106579',
 'https://bi.group/ru/flats?placementUUID=b113b433-614e-11ec-a81f-001dd8b72708&realEstateUUID=c68f11e7-48d1-11eb-a83d-00155d106579',
 'https://bi.group/ru/flats?placementUUID=9db21e85-614e-11ec-a81f-001dd8b72708&realEstateUUID=c68f11e7-48d1-11eb-a83d-00155d106579',
 'https://bi.group/ru/flats?placementUUID=21d2558c-5980-11ec-a829-001dd8b726aa&realEstateUUID=c68f11e7-48d1-11eb-a83d-00155d106579',
 'https://bi.group/ru/flats?placementUUID=19de357c-5980-11ec-a829-001dd8b726aa&realEstateUUID=c68f11e7-48d1-11eb-a83d-00155d106579',
 'https://bi.group/ru/flats?placementUUID=ebf66800-597f-11ec-a829-001dd8b726aa&realEstateUUID=c68f11e7-48d1-11eb-a83d-00155d106579',
 'https://bi.group/ru/flats?placementUUID=078a1225-5980-11ec-a829-001

In [5]:
today = datetime.datetime.today()
today.month+1

10